# LSTM: The basics

In this notebook, we will learn the basics of a Long Short Term Memory (LSTM) based on [Keras](https://keras.io/), a high-level API for building and training deep learning models, running on top of [TensorFlow](https://www.tensorflow.org/), an open source platform for machine learning. 

We will build a basic LSTM to predict stock prices in the future. The data is provided in your training environment - but in future you can also access it in [this github repo](https://github.com/mwitiderrick/stockprice).

### Contents

1. [Converting and preparing data](#convert_data)
2. [A multi-variate LSTM](#simple_lstm)

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

plt.style.use('ggplot')

## Google Cloud Storage Boilerplate

The following two cells have some boilerplate to mount the Google Cloud Storage bucket containing the data used for this notebook to your Google Colab file system. **Even you are not using Google Colab, please make sure you run these two cells.** 

To access the data from Google Colab, you need to:

1. Run the first cell;
2. Follow the link when prompted (you may be asked to log in with your Google account);
3. Copy the Google SDK token back into the prompt and press `Enter`;
4. Run the second cell and wait until the data folder appears.

If everything works correctly, a new folder called `sciml-workshop-data` should appear in the file browser on the left. Depending on the network speed, this may take one or two minutes. Ignore the warning "You do not appear to have access to project ...". If you are running the notebook locally or you have already connected to the bucket, these cells will have no side effects.

In [ ]:
# variables passed to bash; do not change
project_id = 'sciml-workshop'
bucket_name = 'sciml-workshop-data'
colab_data_path = '/content/sciml-workshop-data/'

try:
    from google.colab import auth
    auth.authenticate_user()
    google_colab_env = 'true'
    data_path = colab_data_path + 'sciml-workshop/'
except:
    google_colab_env = 'false'
    ###################################################
    ######## specify your local data path here ########
    ###################################################
    with open('../local_data_path.txt', 'r') as f: data_path = f.read().splitlines()[0]

In [ ]:
%%bash -s {google_colab_env} {colab_data_path} {bucket_name} 

# running locally
if ! $1; then
    echo "Running notebook locally."
    exit
fi

# already mounted
if [ -d $2 ]; then
    echo "Data already mounted."
    exit
fi

apt -qq update
apt -qq install s3fs fuse
mkdir -p $2
s3fs $3 $2 -o allow_other,use_path_request_style,no_check_certificate,public_bucket=1,ssl_verify_hostname=0,host=https://s3.echo.stfc.ac.uk,url=https://s3.echo.stfc.ac.uk

<a id='convert_data'></a>
## Converting data to sequence structure


One of the critical features of $RNNs$ and $LSTMs$ is that they work on sequences of data. In the lecture notes we saw that the network takes input at a time $t$ and the hiden layer state from $t-1$ and produces an output:

<img src="https://github.com/stfc-sciml/sciml-workshop/blob/master/course_3.0_with_solutions/markdown_pic/lstms-1.png?raw=1" alt="lstms-1" width="500"/>

However we may also want to include measured data from further back in time to help with remembering; so we could want input data from $t-1 \cdots t-n$. We might also have more than one channel of input data, this is called the  **window** of the data. Imagine for example we were predicting stock prices, we could have the history of that stock, but we might also want to know the central bank interest rate, or the strength of one currency relative to another, in this case we have a **multi-variate** problem. So our input data looks more like:

<img src="https://github.com/stfc-sciml/sciml-workshop/blob/master/course_3.0_with_solutions/markdown_pic/lstms-2.png?raw=1" alt="lstms-2" width="500"/>


Also, we might also want to make our $LSTM$ predict more than just one step forward, so we will want to be able to have multiple steps in the output.

We write a function to convert dataframe series into data that is suitable for $LSTM$ training. This function is quite flexible and can be useful in many scenarios, so it is one that you might like to reuse in future if you are training time series models.

As input we pass the data as a `numpy` array. We then also specify how far back in time we wish to look for each prediction `n_in`, so `n_in = 1` means we take just $t$ as input, `n_in=2` means we take $t, t-1$ as input and so on. We specify how far into the future we wish to predict, with the `n_out` variable. `n_out=1` means we predict for $t$, `n_out=2` means we predict for $t, t+1$ and so on. In addition to the window sizes, we can also select the features (columns in the data) to use with the `feature_indices` argument. 

The series data can be infinitely long while the memory of our LSTM cannot not be infinitely large. Therefore, we need to specify the length of the LSTM, or the `timesteps`, which is usually much smaller than the total length of the series. The first dimension of the outputs (`X` and `y` for `TensorFlow`) is `batch`; time-dependency is ignored among the batches (i.e., they can be shuffled).

In [ ]:
def series_to_tensorflow(data, timesteps=10, n_in=1, n_out=1, feature_indices=None):
    """
    Convert a series to tensorflow input.
    Arguments:
       data: Sequence of observations as a 2D NumPy array of shape (n_times, n_features)
       n_in: Window size of input X
       n_out: Window size of output y
       feature_indices: select features by indices; pass None to use all features
       timesteps: timesteps of LSTM
    Returns:
       X and y for tensorflow.keras.layers.LSTM
    """
    # sizes
    n_total_times, n_total_features = data.shape[0], data.shape[1]
    n_batches = n_total_times - timesteps - (n_in - 1) - (n_out - 1)
    
    # feature selection
    if feature_indices is None:
        feature_indices = list(range(n_total_features))
    
    # data
    X = np.zeros((n_batches, timesteps, n_in, len(feature_indices)), dtype='float32')
    y = np.zeros((n_batches, n_out, len(feature_indices)), dtype='float32')
    for i_batch in range(n_batches):
        for i_in in range(n_in):
            X_start = i_batch + i_in
            X[i_batch, :, i_in, :] = data[X_start:X_start + timesteps, feature_indices]
        y_start = i_batch + timesteps + n_in - 1
        y[i_batch, :, :] = data[y_start:y_start + n_out, feature_indices]
    
    # flatten the last two dimensions
    return X.reshape(n_batches, timesteps, -1), y.reshape(n_batches, -1)

<a id='simple_lstm'></a>

## A multi-variate LSTM

### Importing and treating the data

We first read in our data and inspect it using `pandas`.


In [ ]:
# Importing the training set
dataset_train = pd.read_csv(data_path + '/lstm-data/data-train-lstm.csv')

In [ ]:
dataset_train.head()

In [ ]:
print(f'Dimensions of raw data: (n_times, n_features) = {dataset_train.shape}')

In the next cell, we get rid of the `Date` column (the first column) and normalize the rest price columns to `[0, 1]` using `MinMaxScaler` from `scikit-learn`.

In [ ]:
# drop `Date` column
values = dataset_train.drop(dataset_train.columns[[0,]], axis=1).values

# normalize prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [ ]:
print(f'Dimensions of normalized price data: (n_times, n_features) = {scaled.shape}')

### Converting to $LSTM$ structure data


We now want to convert the data to a structure that can be fed to the $LSTM$. To do this we use our `series_to_tensorflow` function. In this case, we use 80 as the `timesteps`, looking back 8 steps (`n_in=8`) and projecting forward 4 steps (`n_out=4`) and considering the `Open` and `Close` prices (i.e., `feature_indices=[0, 4]`). 

In [ ]:
timesteps = 80
n_in = 8
n_out = 4
feature_indices = [0, 4]

X_train, y_train = series_to_tensorflow(scaled, timesteps=timesteps, 
                                        n_in=n_in, n_out=n_out, feature_indices=feature_indices)
print(f'Dimensions of X_train: (batches, timesteps, features) = {X_train.shape}')
print(f'Dimensions of y_train: (batches, features) = {y_train.shape}')

### Building the network
<a id='build_lstm'></a>

Note that as before we start off from a `Sequential` type model in `Keras`.

The $LSTM$ layers are already coded in `Keras` so we do not need to worry about writing the complicated structure. We just need to consider a few hyper-parametes we want to set,

* Number of LSTM layers - we can stack LSTM layers in this case we will start with 2 stacked LSTMs;
* units - this is the dimensionality of the hidden state and memory cell of the LSTM;
* return_sequences - should we return the full output sequence or just the final value in the sequence. Generally, if the LSTM layer is feeding to another layer in the network, this would be `True`; if the LSTM layer is the final layer then this is `False`; note default is `False`.

In [ ]:
# Initialising the LSTM
regressor = Sequential()

# Adding the first LSTM layer
# shape: (N, 80, 16) => (N, 80, 50)
regressor.add(LSTM(units=50, return_sequences=True, dropout=0.4,
                   input_shape=(X_train.shape[1], X_train.shape[2])))

# Adding a second LSTM layer
# shape: (N, 80, 50) => (N, 24)
regressor.add(LSTM(units=24, return_sequences=False, dropout=0.4,))

# Adding an output layer 
# shape: (N, 24) => (N, 8)
regressor.add(Dense(units=y_train.shape[1]))

### Compile the network

As ususal we need to compile the network, choosing an optimiser and a loss function. 

We use `adam` as our optimiser and `mean_squared_error` as our loss.

In [ ]:
# Compiling the RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

### Train the network

In [ ]:
# Fitting the RNN to the Training set
history = regressor.fit(X_train, y_train, epochs=30, batch_size=128)

In [ ]:
plt.figure(dpi=100)
plt.plot(history.history['loss'])
plt.xlabel('epoch')
plt.ylabel('mae')
plt.show()

### Making predictions with our model

We now use model that we just built to predict on previously un-seen data. We read in `data-test.csv` and get the stock prices from that data.


In [ ]:
# Part 3 - Making the predictions and visualising the results

# Getting the real stock price of 2017
dataset_test = pd.read_csv(data_path + 'lstm-data/data-test-lstm.csv')
real_stock_price = dataset_test.iloc[:, 1:2].values
print(dataset_test.shape)

In [ ]:
dataset_test.head()

The testing data contain 16 time steps. We will concatenate them to the training data for time extension. 

In [ ]:
# Getting the predicted stock price of 2017
dataset_extended = pd.concat((dataset_train, dataset_test), axis=0)
values_extended = dataset_extended.drop(dataset_extended.columns[[0,]], axis=1).values
scaled_extended = scaler.transform(values_extended)
print(f'Dimensions of normalized price data: (n_times, n_features) = {scaled_extended.shape}')

In [ ]:
# using the last points to generate testing data
X_test, y_test = series_to_tensorflow(scaled_extended[-(timesteps + n_in + n_out - 2 + len(dataset_test)):], 
                                      timesteps=timesteps,
                                      n_in=n_in, n_out=n_out, feature_indices=feature_indices)

# make predictions
y_pred = regressor(X_test)
print(f'Dimensions of X_test: (batches, timesteps, features) = {X_test.shape}')
print(f'Dimensions of y_test: (batches, features) = {y_test.shape}')
print(f'Dimensions of y_pred: (batches, features) = {y_pred.shape}')

In [ ]:
# Visualising the results
fig, ax = plt.subplots(1, len(feature_indices), dpi=100, 
                       figsize=(5 * len(feature_indices), 3), squeeze=False)
for i, index in enumerate(feature_indices):
    # we are ploting the last point from the output window
    ax[0, i].plot(y_test[:, i + (n_out - 1) * len(feature_indices)], 
                  label = 'Real ' + dataset_train.keys()[index + 1])
    ax[0, i].plot(y_pred[:, i + (n_out - 1) * len(feature_indices)], 
                  label = 'Pred ' + dataset_train.keys()[index + 1])
    ax[0, i].legend()
plt.show()

### Exercise

* Build a network with three or four $LSTM$ layers. How does this affect the performance?
* Change the LSTM timesteps (`timesteps`) - how does a longer/shorter one affect the predictions?
* Try other window sizes and features.
